In [47]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
import scipy.spatial.distance as dist

Homework 8.1

In [46]:
hw81 = pd.read_csv('homework_8.1.csv', index_col=0)
hw81.head()

,X,Y,Z
0,1,4.109218,1.764052
1,0,2.259504,0.400157
2,0,-0.647584,0.978738
3,0,2.106071,2.240893
4,1,3.583464,1.867558


In [19]:
X = hw81['X']
Y = hw81['Y']
Z = pd.DataFrame(hw81['Z'])

In [35]:
model = LogisticRegression()
model.fit(Z, X)
prob = model.predict_proba(Z)[:, 1]
prob

array([0.84011371, 0.58464597, 0.71108245, 0.89279265, 0.85308892,
       0.27122817, 0.70536505, 0.45245487, 0.46399577, 0.58709256,
       0.52361497, 0.79574369, 0.66605595, 0.5182238 , 0.59485795,
       0.56897455, 0.80192102, 0.43961717, 0.5640874 , 0.29537176,
       0.07514869, 0.64259816, 0.68788686, 0.31835605, 0.89543145,
       0.19013181, 0.49990255, 0.44389801, 0.8077909 , 0.79810144,
       0.52624106, 0.57947866, 0.28864539, 0.12373123, 0.40598646,
       0.5265785 , 0.75833765, 0.75336345, 0.39684043, 0.41665155,
       0.25783818, 0.19529223, 0.15545602, 0.86287827, 0.36893584,
       0.38517082, 0.22192476, 0.66958077, 0.16753148, 0.43781406,
       0.28712469, 0.58153413, 0.36867663, 0.23419162, 0.48205795,
       0.59123793, 0.50491438, 0.56156954, 0.34137194, 0.40253742,
       0.33313963, 0.40327811, 0.30366873, 0.15293542, 0.53165009,
       0.39350403, 0.16534744, 0.59925363, 0.28479152, 0.50139631,
       0.65915858, 0.51998565, 0.74188884, 0.22493586, 0.58515

In [41]:
weights = np.where(X == 1, 1/prob, 1/(1-prob))
hw81['weights'] = weights
hw81.head()

,X,Y,Z,weights
0,1,4.109218,1.764052,1.190315
1,0,2.259504,0.400157,2.407585
2,0,-0.647584,0.978738,3.461195
3,0,2.106071,2.240893,9.327719
4,1,3.583464,1.867558,1.172211


In [45]:
X_0 = hw81[hw81['X'] == 0]
X_1 = hw81[hw81['X'] == 1]
avg_treated = np.sum(X_1['Y'] * X_1['weights'])/np.sum(X_1['weights'])
avg_untreated = np.sum(X_0['Y'] * X_0['weights'])/np.sum(X_0['weights'])
ate = avg_treated - avg_untreated
ate

np.float64(2.2743411898510133)

Homework 8.2

In [50]:
hw82 = pd.read_csv('homework_8.2.csv', index_col=0)
hw82.head()

,X,Y,Z1,Z2
0,1,4.652085,1.764052,2.320015
1,1,2.590221,0.400157,1.292631
2,1,3.898981,0.978738,0.556423
3,1,5.857179,2.240893,2.345607
4,1,3.647489,1.867558,2.095611


In [66]:
treated = hw82[hw82['X'] == 1].reset_index(drop=True)
untreated = hw82[hw82['X'] == 0].reset_index(drop=True)

In [56]:
matrix = np.vstack((hw82['Z1'], hw82['Z2']))
matrix

array([[ 1.76405235,  0.40015721,  0.97873798, ...,  0.0941923 ,
        -1.14761094, -0.35811408],
       [ 2.32001503,  1.2926311 ,  0.55642316, ...,  0.25262615,
        -2.28951236, -1.66908445]], shape=(2, 1000))

In [61]:
cov = np.cov(matrix)
cov_inv = np.linalg.inv(cov)
cov_inv

array([[ 2.02734407, -1.03387633],
       [-1.03387633,  1.06684813]])

In [79]:
for i, row in treated.iterrows():
    treated_z = row[['Z1', 'Z2']].values

In [69]:
matches = []
for i, row in treated.iterrows():
    treated_z = row[['Z1', 'Z2']].values
    distances = untreated[['Z1', 'Z2']].apply(lambda x: dist.mahalanobis(treated_z, x.values, cov_inv), axis=1)
    nearest_index = distances.idxmin()
    matches.append((i, nearest_index))

matched_pairs = pd.DataFrame([{'treated_index': t_idx, 'control_index': c_idx, 'treated_Y': treated.loc[t_idx,'Y'], 'control_Y': untreated.loc[c_idx, 'Y']} for t_idx, c_idx in matches])
matched_pairs

,treated_index,control_index,treated_Y,control_Y
0,0,352,4.652085,0.428954
1,1,392,2.590221,-0.034844
2,2,466,3.898981,1.164988
3,3,481,5.857179,1.797450
4,4,481,3.647489,1.797450
...,...,...,...,...
478,478,475,1.763565,-2.545817
479,479,464,3.871122,1.191166
480,480,4,1.394735,-0.426837
481,481,195,5.336848,0.256826


In [70]:
matched_pairs['TE'] = matched_pairs['treated_Y'] - matched_pairs['control_Y']
matched_pairs.head()

,treated_index,control_index,treated_Y,control_Y,TE
0,0,352,4.652085,0.428954,4.223131
1,1,392,2.590221,-0.034844,2.625065
2,2,466,3.898981,1.164988,2.733993
3,3,481,5.857179,1.797450,4.059729
4,4,481,3.647489,1.797450,1.850039


In [71]:
np.mean(matched_pairs['TE'])

np.float64(3.437678997912609)

In [72]:
matched_pairs['distance'] = matched_pairs.apply(lambda row: dist.mahalanobis(treated.loc[row['treated_index'], ['Z1', 'Z2']].values, untreated.loc[row['control_index'], ['Z1', 'Z2']].values, cov_inv), axis=1)
worst_match = matched_pairs.loc[matched_pairs['distance'].idxmax()]
worst_treated = treated.loc[worst_match['treated_index'], ['Z1', 'Z2']]
worst_treated

Z1    2.696224
Z2    0.538155
Name: 241, dtype: float64